In [1]:
import nbimporter
import read_report_cobol as cobol_reader # type: ignore
import read_report_java as java_reader # type: ignore

cobol = cobol_reader.get_cobol_data_from("RHPP05LA.PDF")
java = java_reader.get_java_data_from("RHPP05LA - GERAL.pdf")


abr 10, 2024 4:54:22 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
ADVERTÊNCIA: Using fallback font LiberationSans for base font Symbol
abr 10, 2024 4:54:22 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
ADVERTÊNCIA: Using fallback font LiberationSans for base font ZapfDingbats


In [2]:
code = 40434

display("cobol", "clt", "apropriado")
display(cobol[code]['clt']['apropriado'])
display("realizado")
display(cobol[code]['clt']['realizado'])
display("autarquico", "apropriado")
display(cobol[code]['autarquico']['apropriado'])
display("realizado")
display(cobol[code]['autarquico']['realizado'])
display("java", "clt", "apropriado")
display(java[code]['clt']['apropriado'])
display("realizado")
display(java[code]['clt']['realizado'])
display("autarquico", "apropriado")
display(java[code]['autarquico']['apropriado'])
display("realizado")
display(java[code]['autarquico']['realizado'])

'cobol'

'clt'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,360940.90,120313.76,481254.66,360940.90,842195.56
1,inss,61317.13,20439.04,81756.17,61317.13,143073.30
2,ac_trab,1531.42,510.10,2041.52,1531.42,3572.94
3,fgts,24526.11,8175.33,32701.44,24526.11,57227.55
4,spprevcom,0.00,0.00,0.00,1668.03,1668.03
5,total_encargos,87374.66,29124.47,116499.13,89042.69,205541.82
6,total,448315.56,149438.23,597753.79,449983.59,1047737.38


,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,395869.72,131954.37,527824.09,13408.63,541232.72
1,inss,73436.62,24477.13,97913.75,2630.23,100543.98
2,ac_trab,1841.74,611.38,2453.12,65.66,2518.78
3,fgts,29549.08,9847.95,39397.03,1052.06,40449.09
4,spprevcom,0.00,0.00,0.00,0.00,0.00
5,total_encargos,104827.44,34936.46,139763.90,3747.95,143511.85
6,total,500697.16,166890.83,667587.99,17156.58,684744.57


'autarquico'

...

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,364641.26,121545.68,486186.94,364641.26,850828.20
1,inss,72926.76,24308.00,97234.76,62057.50,159292.26
2,ac_trab,182319.78,60771.96,243091.74,155146.80,398238.54
3,fgts,29169.32,9721.89,38891.21,24822.20,63713.41
4,spprevcom,0.00,0.00,0.00,1959.04,1959.04
5,total_encargos,284415.86,94801.85,379217.71,243985.54,623203.25
6,total,649057.12,216347.53,865404.65,608626.80,1474031.45


,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,491626.71,164915.44,656542.15,562.31,657104.46
1,ferias_pecunia,105434.36,0.00,105434.36,0.00,105434.36
2,total_ferias,597061.07,164915.44,761976.51,562.31,762538.82
3,inss,119411.83,32982.72,152394.55,0.00,152394.55
4,ac_trab,298530.29,82457.55,380987.84,0.00,380987.84
5,fgts,47764.42,13192.79,60957.21,0.00,60957.21
6,spprevcom,0.00,0.00,0.00,0.00,0.00
7,total_encargos,465706.54,128633.06,594339.60,0.00,594339.60
8,total,1062767.61,293548.50,1356316.11,562.31,1356878.42


'autarquico'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,261645.95,87214.79,348860.74,261645.95,610506.69
1,spprevcom,0.00,0.00,0.00,164.54,164.54
2,total,261645.95,87214.79,348860.74,261810.49,610671.23
3,spprev,0.00,0.00,0.00,376284.78,376284.78


,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,146256.84,49705.68,195962.52,0.00,195962.52
1,spprevcom,0.00,0.00,0.00,0.00,0.00
2,total,146256.84,49705.68,195962.52,0.00,195962.52
3,spprev,0.00,0.00,0.00,0.00,0.00
